In [1]:
import pandas as pd
import data_util
from tec_an import TecAn
import numpy as np
from data_util import *
from bitstamp import *
from sklearn_model_hyper import *
from joblib import dump, load
from data_generator import *
from data_agent import *
from stock_agent import *
from backtest import *

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer
from sklearn.neural_network import MLPClassifier


In [2]:
live = LiveTrader()

In [3]:
live.init()

In [4]:
live.user_transactions(3)

[]

In [5]:
def get_online_data(minutes, source_data_generator, load_from_disk, file_prefix = ""):
    
    online = OnLineDataProvider(
                 source_data_generator = source_data_generator,
                 minutes = minutes,
                 train_keys = [],
                 train_limit = 1000,
                 val_limit = 1000,
                 val_keys = ["btcusd"],
                 val_start = 1234,
                 val_end = 1234,
                 train_start_list = []
    )

    online_path = f'data/online{file_prefix}_{minutes}'
    
    if (load_from_disk):
        online = load(online_path)    
    else:
        #online.load_train_cache()
        online.load_cache()
        online.sourceDataGenerator = None
        dump(online, online_path)
        
    
    return online

In [6]:
cls = make_pipeline( StandardScaler(),
        MLPClassifier(
            solver='lbfgs', alpha=1, random_state=1, max_iter=20000,
            early_stopping=True)
        )

In [19]:
tec = TecAn(windows = 30, windows_limit = 100)
sourceDataGenerator = SourceDataGenerator(tec = tec)
online = get_online_data(15, sourceDataGenerator, True, 30)

In [20]:
trainX_raw, trainY_raw = online.load_train_data()
x, y, closed_prices = prepare_train_data(trainX_raw, trainY_raw, 1)

In [100]:
def get_y_data(ohlc, shift = -1):
    combined_data = ohlc.copy()
    #combined_data['return'] = np.log(combined_data / combined_data.shift(1))
    
    keys = []
    #print(f"##### {shift}")
    steps = (shift * -1) + 1
    for idx in range(1, steps):
        returns = (ohlc / ohlc.shift(-1 * idx))
        key = f'{idx}'
        keys.append(key)
        combined_data[key] = returns
    
    for key in keys:
        combined_data[f'direction{key}'] = np.where(combined_data[key] < 0.997, 1, 0)
    
    combined_data[f'direction'] = combined_data[f'direction{keys[0]}']
    for idx in range(1, len(keys)):
        combined_data[f'direction'] = combined_data[f'direction{keys[idx]}'] + combined_data[f'direction'] 
    
    combined_data[f'y'] = np.where(combined_data['direction'] > 0, 1, 0)
    
    print(combined_data)
    
    return combined_data[f'y'].to_numpy()

In [101]:
close = pd.DataFrame(trainY_raw, columns = ['Close'])

In [102]:
yy = get_y_data(close, -1)

         Close         1  direction1  direction  y
0     32625.67  1.006118           0          0  0
1     32427.29  0.995271           1          1  1
2     32581.37  0.999744           0          0  0
3     32589.70  0.996903           1          1  1
4     32690.95  0.992127           1          1  1
...        ...       ...         ...        ... ..
7987  37085.33  0.998571           0          0  0
7988  37138.41  1.000665           0          0  0
7989  37113.72  0.998580           0          0  0
7990  37166.48  1.004472           0          0  0
7991  37001.01       NaN           0          0  0

[7992 rows x 5 columns]


In [103]:
print(Counter(yy))

Counter({0: 6831, 1: 1161})


In [99]:
print(Counter(yy))

Counter({0: 6303, 1: 1689})


In [8]:
train_by_step(cls, 1, online)

(array([[ 1.64137365e+00,  4.03211107e-01, -7.86578302e+00, ...,
         -1.24449017e-01, -8.68977673e-01, -1.33274854e+00],
        [ 4.01597033e-01,  3.35500840e-01, -2.18505879e+00, ...,
         -1.48594286e-01, -8.71896015e-01, -6.95664504e+00],
        [-7.21481523e+00,  3.15081370e-01, -1.40318415e+01, ...,
         -4.70509068e-01, -2.81936268e+00, -2.24095468e+00],
        ...,
        [-2.43097014e-01,  4.85940776e-01, -1.87323996e+00, ...,
         -1.12681688e-02, -1.42672242e-01,  9.44619423e-01],
        [-1.58798986e+02,  2.14454952e-01,  2.85568570e+00, ...,
         -6.46519104e-01, -2.98552843e+00, -4.08735392e+00],
        [ 5.78439448e+00,  5.42046109e-01, -7.93549213e-01, ...,
          4.43062509e-02,  5.09401809e-01, -3.08612901e-01]]),
 array([0, 1, 0, ..., 0, 1, 1]),
 array([ 3374.57,  9448.99,  9859.86, ...,  7691.98, 37457.49,  9319.52]))

In [10]:
back = BackTest(value = 100,
                        verbose = True, 
                        sell_on_profit = True)

In [11]:
request_sell = lambda price: back.request_sell(price)
request_buy = lambda price: back.request_buy(price)
on_state = lambda timestamp, price: back.on_state(timestamp, price)

In [12]:
on_state(0, 3)
request_buy(10)

1970-01-01 00:00:00 Buy (3): ask: 10


In [13]:

request_sell(10)


#### LOSSSS: 0.0
1970-01-01 00:00:00 Sell (3) profit: 0.0 total: 100.0
